In [1]:
from platform import python_version

print(python_version())

3.7.11


In [2]:
import sys
sys.executable

'C:\\Users\\Ben\\anaconda3\\python.exe'

In [2]:
import os
import sys
#connect local PATH
p = os.path.abspath(r'C:\Users\Ben\Desktop\Diplomatiki\CryptoSent\crypto_sent_tools')
sys.path.insert(1, p)


In [3]:
import crypto_sent_tools as cs




In [ ]:
cs.c

In [1]:
import pandas as pd
import os
import sys
#connect local PATH
p = os.path.abspath(r'C:\Users\Ben\Desktop\Diplomatiki\CryptoSent\utils')
sys.path.insert(1, p)

import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [2]:
c=pd.read_pickle(r"C:\Users\Ben\Desktop\Diplomatiki\CryptoSent\Data Preparation\3_Data_Processing\top500k_comments.pickle")
s=pd.read_pickle(r"C:\Users\Ben\Desktop\Diplomatiki\CryptoSent\Data Preparation\3_Data_Processing\TOP20K_submissions.pickle")

In [11]:

subid_comments=c.groupby('submission_id').body.sum().to_dict()

subid_com_score=c.groupby('submission_id').score.sum().to_dict()

In [19]:
s['comment_score']=s.id.map(subid_com_score)
s['comment_score']=s['comment_score'].fillna(0).astype(int)

In [20]:
s['comments']=s.id.map(subid_comments)

In [27]:
df=s[~s.comments.isna()]

In [28]:
df.to_pickle('main_merge_v1.pickle')

## running 

In [2]:
df=pd.read_pickle('main_merge_v1.pickle')

In [4]:
df.head(5).T

,1474281,1400525,1547559,964610,935676
created,2021-05-19 02:00:14,2021-05-08 17:28:35,2021-05-31 12:01:31,2021-02-08 13:48:45,2021-02-04 13:02:16
author,AutoModerator,jonbristow,jasonluxton,AlphaGrayWolf,[deleted]
num_comments,58597,4303,7539,2954,3659
score,1013,52190,43785,45445,39951
title,"Daily Discussion - May 19, 2021 (GMT+0)",You hear about the kid who put in $500 into a ...,"Binance CEO, CZ, shades Elon Musk in tweet. - ...",Tesla buys $1.5b in Bitcoin and is looking to ...,Elon Musk reminds me of a rich billionaire who...
selftext,**Welcome to the Daily Discussion. Please read...,You hear about the kid who put in $500 into a ...,NaN,NaN,[deleted]
subreddit,CryptoCurrency,CryptoCurrency,CryptoCurrency,Bitcoin,CryptoCurrency
subreddit_subscribers,3092129,3091100,3093308,3070878,3082709
id,nfqb4c,n7rl2y,noztp7,lfb3mm,lce6zm
domain,self.CryptoCurrency,self.CryptoCurrency,coinquora.com,v.redd.it,self.CryptoCurrency


In [59]:
import re
urlremove = lambda x: re.sub(r'http\S+', '', x)

In [72]:
df['all_text']=df.title.fillna(' ')+' \\'+df.selftext.fillna(' ')+' \\'+df.comments.fillna(' ')

In [74]:
df.all_text=df.all_text.apply(urlremove)

In [80]:
import nltk
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()
from nltk.corpus import stopwords
cached_stopwords=stopwords.words('english')
def clean_text(text):
    text=regex_magic(text).lower()
    text= re.sub('[0-9]+', '', text)#removed numbers
    tokens = re.split('\W+', text)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in cached_stopwords]  # remove stopwords and stemming
    text = [wn.lemmatize(word) for word in text]
    return text
from tqdm import tqdm
tqdm.pandas()

regex_magic= lambda x: ' '.join(re.sub("(#[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
# Importing the dataset
sent140 = pd.read_csv(r"D:\DTU\Semester 3 - Fall 2020\Social Graphs and Interactions\__Final Project__\Data Proofs\trainingandtestdata\sentiment140.csv",
                      encoding="ISO-8859-1" , names=["sentiment", "ids", "date", "flag", "user", "text"])
#taking the columns that we need
sent140 = sent140[['sentiment','text']]
sent140['sentiment'] = sent140['sentiment'].replace(4,1)
#preprocessing the text
sent140['text_clean']=sent140.text.progress_apply(clean_text)
sent140['text_clean_join']=sent140.text_clean.progress_apply(lambda x: ' '.join(x))

100%|███████████████████████████████████████████████████████████████████| 1600000/1600000 [00:01<00:00, 1128661.40it/s]


In [82]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sent140['text_clean_join'],
                                                    sent140.sentiment,
                                                    test_size = 0.05, random_state = 42)
word2vec = TfidfVectorizer(ngram_range=(1,2), max_features=50000)
word2vec.fit(X_train)
X_train = word2vec.transform(X_train)
X_test  = word2vec.transform(X_test)

In [83]:
import _pickle as cPickle
# load it again
with open(r"C:\Users\Ben\Desktop\Diplomatiki\CryptoSent\Sentiment Models\my_dumped_classifier.pkl", 'rb') as fid:
    lr_clf = cPickle.load(fid)

In [84]:
# Getting the sentiment of the Tweets
def get_sentiment(text):
    text=word2vec.transform([text])
    neg,pos=lr_clf.predict_proba(text)[0]
    if neg>.6:
        sentiment='Negative'
    elif pos>.6:
        sentiment='Positive'
    else:
        sentiment='Netural'
    return neg,pos,sentiment

list2doc=lambda x: ' '.join(x)

In [97]:
df['text_processed']=df['all_text'].progress_apply(lambda x: list2doc(clean_text(x)))

100%|████████████████████████████████████████████████████████████████████████████| 18408/18408 [03:26<00:00, 89.23it/s]


In [98]:
df['sent_dict']=df.all_text.progress_apply(get_sentiment)

100%|███████████████████████████████████████████████████████████████████████████| 18408/18408 [00:38<00:00, 474.36it/s]


In [99]:
# claculating and normalizing the polarity or in other words the confidence of the model's prediction
pos=df.sent_dict.apply(lambda x: x[1]).tolist()
neg=df.sent_dict.apply(lambda x: x[0]).tolist()
pos_max=max(pos)
pos_min=min(pos)
def normalize_polarity(x):
    avg=(pos_max+pos_min)/2
    ran=(pos_max-pos_min)/2
    return (x-avg)/ran

df['polarity']=df.sent_dict.progress_apply(lambda x: normalize_polarity(x[1]))
df['sentiment']=df.sent_dict.apply(lambda x: x[2])

100%|████████████████████████████████████████████████████████████████████████| 18408/18408 [00:00<00:00, 642950.81it/s]


In [106]:
df.index=pd.to_datetime(df.created)

# Comparison

In [90]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [174]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [248]:
dchunk='m'
a=df.resample(dchunk).sum().polarity
btc_price=pd.read_csv(r"C:\Users\Ben\Desktop\Diplomatiki\CryptoSent\Datasets\other\coin_Bitcoin.csv")
btc_price.Date=pd.to_datetime(btc_price.Date)
btc_price=btc_price.set_index('Date')
btc_price['avg_price']=(btc_price.High+btc_price.Low)/2

btc_price_avg=btc_price.resample(dchunk).mean().loc['2019':'2021-06']['avg_price']

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=a.index, y=a, name="LR-Model"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=btc_price_avg.index, y=btc_price_avg, name="Bit Coin Avg"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="BTC vs LR Sentiment Model - Weekly"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Sentiment Polarity </b>", secondary_y=False)
fig.update_yaxes(title_text="<b>BTC Price</b>", secondary_y=True)

fig.show()

In [286]:
dchunk='d'
a=df.resample(dchunk).sum().polarity
btc_price=pd.read_csv(r"C:\Users\Ben\Desktop\Diplomatiki\CryptoSent\Datasets\other\coin_Bitcoin.csv")
btc_price.Date=pd.to_datetime(btc_price.Date)
btc_price=btc_price.set_index('Date')
btc_price['avg_price']=(btc_price.High+btc_price.Low)/2

btc_price_avg=btc_price.resample(dchunk).mean().loc['2019':'2021-06']['avg_price']
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=a.index, y=a, name="LR-Model"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=btc_price_avg.index, y=btc_price_avg, name="Bit Coin Avg"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="BTC vs LR Sentiment Model - Weekly"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Sentiment Polarity </b>", secondary_y=False)
fig.update_yaxes(title_text="<b>BTC Price</b>", secondary_y=True)

fig.show()

In [289]:
dchunk='w'
a=df.resample(dchunk).sum().polarity
btc_price=pd.read_csv(r"C:\Users\Ben\Desktop\Diplomatiki\CryptoSent\Datasets\other\coin_Bitcoin.csv")
btc_price.Date=pd.to_datetime(btc_price.Date)
btc_price=btc_price.set_index('Date')
btc_price['avg_price']=(btc_price.High+btc_price.Low)/2

btc_price_avg=btc_price.resample(dchunk).mean().loc['2019':'2021-06']['avg_price']

apct=a.pct_change().tail(-1)


apct=apct['2019':'2021-06']

btcpct=btc_price_avg.pct_change().tail(-1)

In [290]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=apct.index, y=apct, name="LR-Model"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=btcpct.index, y=btcpct, name="Bit Coin Avg"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="BTC vs LR Sentiment Model - Weekly"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Sentiment Polarity </b>", secondary_y=False)
fig.update_yaxes(title_text="<b>BTC Price</b>", secondary_y=True)

fig.show()

In [291]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=btc_price_avg.diff().index, y=btc_price_avg.diff(), name="LR-Model"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=a.index, y=a, name="Bit Coin Avg"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="BTC vs LR Sentiment Model - Weekly"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Sentiment Polarity </b>", secondary_y=False)
fig.update_yaxes(title_text="<b>BTC Price</b>", secondary_y=True)

fig.show()

In [261]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=btc_price_avg.diff().index, y=btc_price_avg.diff(), name="LR-Model"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=a.index, y=a, name="Bit Coin Avg"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="BTC vs LR Sentiment Model - Weekly"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Sentiment Polarity </b>", secondary_y=False)
fig.update_yaxes(title_text="<b>BTC Price</b>", secondary_y=True)

fig.show()

In [263]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=btc_price_avg.pct_change().index, y=btc_price_avg.pct_change(), name="LR-Model"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=a.index, y=a, name="Bit Coin Avg"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="BTC vs LR Sentiment Model - Weekly"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Sentiment Polarity </b>", secondary_y=False)
fig.update_yaxes(title_text="<b>BTC Price</b>", secondary_y=True)

fig.show()

In [367]:
dchunk='d'
a=df[df.subreddit=='Bitcoin'].resample(dchunk).sum().polarity
btc_price=pd.read_csv(r"C:\Users\Ben\Desktop\Diplomatiki\CryptoSent\Datasets\other\coin_Bitcoin.csv")
btc_price.Date=pd.to_datetime(btc_price.Date)
btc_price=btc_price.set_index('Date')
btc_price['avg_price']=(btc_price.High+btc_price.Low)/2

btc_price_avg=btc_price.resample(dchunk).mean().loc['2020':'2021-06']['avg_price']

apct=a.pct_change().tail(-1)


apct=apct['2021':'2021-06']

btcpct=btc_price_avg.pct_change().tail(-1)

In [368]:
x=btc_price_avg.tail(-1)

In [369]:
y=a['2020':'2021-06'].tail(-1)

In [370]:
import plotly.express as px
fig = px.scatter(x=x.pct_change().tolist(), y=y.tolist())
fig.show()

fig = px.scatter(x=x.diff().tolist(), y=y.tolist())
fig.show()

